# Soprano saxophone

* Sample duration = 5s
* model = SVM
* target = SOP

Distinguish between samples with soprano saxophone from those without soprano saxophone. Other brass and (most) woodwind instruments excluded.

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, \
    RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import uniform

# from pickles_to_pandas import pickles_to_pandas


In [4]:
# df = pickles_to_pandas('./data/5s/labeled/features_r02')
df = pd.read_pickle('./data/5s/labeled/features_r02/all_data.pkl')

In [26]:
NUM_LABEL_COLS = 13
NUM_COMPONENTS = 20
RANDOM_SEED = 0

In [6]:
# df_filtered = df[df['excl'] == '0']  # exclude records we want to exclude
df_filtered = df
df_filtered.shape

(6819, 10790)

In [7]:
# exclude records we want to exclude
df_filtered = df_filtered[df_filtered['alto'] == '0'] 
df_filtered = df_filtered[df_filtered['tenr'] == '0']
df_filtered = df_filtered[df_filtered['bari'] == '0']
df_filtered = df_filtered[df_filtered['clrt'] == '0']
df_filtered = df_filtered[df_filtered['tora'] == '0']
df_filtered = df_filtered[df_filtered['othr'] == '0']
df_filtered = df_filtered[df_filtered['trmp'] == '0']
df_filtered = df_filtered[df_filtered['trmb'] == '0']
df_filtered = df_filtered[df_filtered['otrb'] == '0']

num_x_cols = df_filtered.shape[1] - NUM_LABEL_COLS - 1  
# ^ last bit to adjust for zero indexing
data = df_filtered.iloc[:, 1:num_x_cols].to_numpy() 
# ^ These are the features
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

target = df_filtered[['sop']].to_numpy().ravel()  # << This is the label

print(data.shape)
print(target.shape)

(2348, 10775)
(2348,)


In [10]:
pca = PCA(n_components=NUM_COMPONENTS)

In [11]:
pca.fit(data)

PCA(n_components=20)

In [12]:
# print(pca.explained_variance_ratio_)

In [13]:
# print(pca.singular_values_)

In [14]:
d = pca.transform(data)

In [15]:
# d

In [16]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target, test_size=0.20, random_state=0)

## Test plain vanilla SVM

In [17]:
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.951063829787234


In [18]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,438,0
1,23,9


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       438
           1       1.00      0.28      0.44        32

    accuracy                           0.95       470
   macro avg       0.98      0.64      0.71       470
weighted avg       0.95      0.95      0.94       470



## Determine good value for C using random search

In [20]:
model_params = {
    'C': uniform(0.1, 3.0)
}

# RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, 
#                    scoring=None, n_jobs=None, iid='deprecated', 
#                    refit=True, cv=None, verbose=0, 
#                    pre_dispatch='2*n_jobs', random_state=None, 
#                    error_score=nan, return_train_score=False)
svm_model = SVC()
clf = RandomizedSearchCV(svm_model, model_params, n_iter=10, 
                         cv=5)
model = clf.fit(x_train, y_train)
params = model.best_estimator_.get_params()

In [21]:
clf = SVC(C=params['C'])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.9638297872340426


In [22]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,438,0
1,17,15


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       438
           1       1.00      0.47      0.64        32

    accuracy                           0.96       470
   macro avg       0.98      0.73      0.81       470
weighted avg       0.97      0.96      0.96       470



## Test ensemble approach with bagging classifier

In [25]:
# Parameters
p = {'n_estimators': [100, 200, 300, 400, 500],
     'max_features': [NUM_COMPONENTS - 2, NUM_COMPONENTS - 3, 
                      NUM_COMPONENTS - 4, NUM_COMPONENTS - 5,
                      NUM_COMPONENTS - 6, NUM_COMPONENTS - 7]}

base_estimator = SVC(C=params['C'])
estimator = BaggingClassifier(base_estimator=base_estimator)
# Question: Should we include C here, or ... ? There's no reason it should differ
# for different values of n_estimators, but what about max_features? Might different
# values for C be best for different max_features? Maybe. Dunno. Maybe we should
# search for optimal C *after* we've found the best value for max_features?

# GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, 
#              iid='deprecated', refit=True, cv=None, verbose=0, 
#              pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
clf = GridSearchCV(estimator, p, n_jobs=-1)
search = clf.fit(x_train, y_train)
search.best_params_

# {'max_features': 18, 'n_estimators': 100}

{'max_features': 18, 'n_estimators': 100}

In [27]:
# BaggingClassifier(base_estimator=None, n_estimators=10, *, 
#                   max_samples=1.0, max_features=1.0, 
#                   bootstrap=True, bootstrap_features=False, 
#                   oob_score=False, warm_start=False, 
#                   n_jobs=None, random_state=None, verbose=0) 
clf = BaggingClassifier(base_estimator=SVC(C=params['C']),
                        n_estimators=search.best_params_['n_estimators'], 
                        max_features=search.best_params_['max_features'],
                        random_state=RANDOM_SEED)
model = clf.fit(x_train, y_train)

In [28]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.9617021276595744


In [29]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,438,0
1,18,14


In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       438
           1       1.00      0.44      0.61        32

    accuracy                           0.96       470
   macro avg       0.98      0.72      0.79       470
weighted avg       0.96      0.96      0.95       470

